# Forcasting

```plaintext

DEFINE FUNCTION compute_accuracy WITH parameter row
    IF 'RATE_actual' VALUE in row IS 0 THEN
        RETURN Not a Number
    ELSE
        CALCULATE absolute difference BETWEEN 'RATE_actual' AND 'RATE_forecasted' in row, STORE IN error
        CALCULATE error proportion AS error DIVIDED BY 'RATE_actual' in row
        RETURN (1 MINUS error_proportion) MULTIPLIED BY 100
    END IF
END FUNCTION

SET weeks AS 12
FORECAST next 12 weeks USING ARIMA model AND STORE IN forecasted_values

# Ensure that 'YearMonthWeek' is a datetime object
CONVERT 'YearMonthWeek' column IN agg_df TO datetime type
FIND last_date AS last element IN 'YearMonthWeek' column OF agg_df
CREATE forecasted_dates AS date range FROM last_date TO weeks+1 WITH frequency WEEKLY, EXCLUDE first date

CREATE df_forecasted DataFrame FROM {
    'POD': forecasted_dates,
    'RATE': reshaped forecasted_values
}

ROUND 'RATE' column IN df_forecasted TO 2 decimal places

DISPLAY first 5 rows of df_forecasted
DISPLAY last 5 rows of df_forecasted
DISPLAY information of df_forecasted

```

# Accuracy Comparison

```plaintext
CREATE comparison_df AS DataFrame WITH columns 'WeekStart', 'WeekEnd', 'POD_actual', 'RATE_forecasted', 'RATE_actual'
ADD 7 days TO 'POD' column in df_forecasted AND STORE in new column 'WeekEnd'

FOR each row IN df_forecasted
    CREATE mask WHERE 'POD' column of new_dates_df IS GREATER THAN OR EQUAL TO 'POD' column in current row AND 'POD' column of new_dates_df IS LESS THAN 'WeekEnd' column in current row
    GET actual_dates_within_week FROM new_dates_df WHERE mask IS TRUE
    
    FOR each actual_row IN actual_dates_within_week
        APPEND {'WeekStart': current row's 'POD', 'WeekEnd': current row's 'WeekEnd', 'POD_actual': actual row's 'POD', 'RATE_forecasted': current row's 'RATE', 'RATE_actual': actual row's 'RATE'} TO comparison_df
    END FOR
END FOR

REMOVE duplicates FROM comparison_df BASED ON subset ['POD_actual', 'RATE_forecasted', 'RATE_actual'] AND RESET index

# Compute accuracy
ADD 'accuracy' column TO comparison_df APPLYING compute_accuracy FUNCTION across each row
REMOVE rows WITH 'NaN' values in 'accuracy' column FROM comparison_df

CALCULATE mean of 'accuracy' column of comparison_df AND STORE in total_mean_accuracy
PRINT total_mean_accuracy

```